In [10]:
import numpy as np
import collections 
import random as rd


In [70]:
class puzzle:
    
    board = None
    parent = None
    depth = None
    
    def __init__(self,board,parent=None,depth=0):
        self.board = board
        self.parent = parent
        self.depth = depth


In [79]:
def CreaEstado():
    board = np.zeros((4,4))
    randpos = (rd.randint(0,3),rd.randint(0,3))
    board[randpos] = 2
    
    return puzzle(board)
    
startObj = CreaEstado()
print startObj.board

[[0. 0. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
